# --- Day 19: Medicine for Rudolph ---
https://adventofcode.com/2015/day/19

In [26]:
from collections import defaultdict
import re
import heapq

def getMolecule():
    with open("molecule.txt") as file:
        return file.read()
    
def getReplacements():
    with open("replacements.txt") as file:
        return file.read()

In [27]:
molecule = getMolecule()
replacements = getReplacements().split("\n")

# Map the replacements
replacementsMap = defaultdict(list)
for fromMolecule, toMolecule in [x.split(" => ") for x in replacements]: 
	replacementsMap[fromMolecule].append(toMolecule)

def replaceMolecule(start: int, end: int, newMolecule: str, molecule: str) -> str:
	"""Replace the molecule in the range with new molecule"""
	moleculeBefore = molecule[:start]
	moleculeAfter = molecule[end:]
	return moleculeBefore + newMolecule + moleculeAfter

# Loop through all possible replacements
allNewMolecules = []
for moleculeToReplace in replacementsMap.keys():
	# Find all indexes where that molecule occurs
	allStartIndexes = [[x.start(), x.end()] for x in re.finditer(moleculeToReplace, molecule)]
	# For each occurrence, replace it with each possible replacement
	for start, end in allStartIndexes:
		for replacement in replacementsMap[moleculeToReplace]:
			allNewMolecules.append(replaceMolecule(start, end, replacement, molecule))

print(f"Number of distinct molecules with one replacement: {len(set(allNewMolecules))}")

Number of distinct molecules with one replacement: 509


# --- Part Two ---

In [ ]:
targetMolecule = getMolecule()
replacements = getReplacements().split("\n")

class Molecule:
	def __init__(self, molecule: str, steps: int):
		self.molecule = molecule
		self.steps = steps
		self.score = self.getScore()

	def getScore(self):
		score = 0
		# Get the number of characters that are in the right position
		for i in range(min(len(self.molecule), len(targetMolecule))):
			if self.molecule[i] == targetMolecule[i]:
				score += 1

		# Incentivise being closer to the correct length
		score -= abs(len(self.molecule) - len(targetMolecule))
		return score
	
	def __lt__(self, other):
		# Intentionally backwards since we'll be using a min heap when we really want a max heap
		return self.score > other.score

# Map the replacements
replacementsMap = defaultdict(list)
for fromMolecule, toMolecule in [x.split(" => ") for x in replacements]: 
	replacementsMap[fromMolecule].append(toMolecule)

def replaceMolecule(start: int, end: int, newMolecule: str, molecule: str) -> str:
	"""Replace the molecule in the range with new molecule"""
	moleculeBefore = molecule[:start]
	moleculeAfter = molecule[end:]
	return moleculeBefore + newMolecule + moleculeAfter

def getMoleculeSuccessors(currentMolecule: str) -> list[str]:
	# Loop through all possible replacements
	allNewMolecules = []
	for moleculeToReplace in replacementsMap.keys():
		# Find all indexes where that molecule occurs
		allStartIndexes = [[x.start(), x.end()] for x in re.finditer(moleculeToReplace, currentMolecule)]
		# For each occurrence, replace it with each possible replacement
		for start, end in allStartIndexes:
			for replacement in replacementsMap[moleculeToReplace]:
				allNewMolecules.append(replaceMolecule(start, end, replacement, currentMolecule))
	return allNewMolecules

def aStar(startMolecule: str, targetMolecule: str) -> int:
	"""Return the number of steps it takes to get from startMolecule to targetMolecule"""
	molequeue = [Molecule(molecule=startMolecule, steps=0)] # Get it? Molequeue! Like molecule and queue!
	explored = set(startMolecule)

	# Continue until queue is empty or until we find the targetMolecule
	while molequeue:
		# Gets the next highest scoring molecule
		currentMolecule = heapq.heappop(molequeue)

		# get successors and add to queue if not already explored
		for successor in getMoleculeSuccessors(currentMolecule=currentMolecule.molecule):
			if successor == targetMolecule:
				return currentMolecule.steps + 1
			if successor not in explored:
				# Add new molecule to explored and the priority molequeue
				explored.add(successor)
				heapq.heappush(molequeue, Molecule(molecule=successor, steps=currentMolecule.steps + 1))

aStar(startMolecule="e", targetMolecule=targetMolecule)